In [13]:
%load_ext autoreload
%autoreload 2

import qm9.visualizer as vis
from eval_sample import *
import argparse
import warnings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_path', type=str,
                    default="outputs/edm_qm9",
                    help='Specify model path')
parser.add_argument(
    '--n_tries', type=int, default=10,
    help='N tries to find stable molecule for gif animation')
parser.add_argument('--n_nodes', type=int, default=19,
                    help='number of atoms in molecule for gif animation')

eval_args, unparsed_args = parser.parse_known_args()

assert eval_args.model_path is not None

print(join(eval_args.model_path, 'args.pickle'))

with open(join(eval_args.model_path, 'args.pickle'), 'rb') as f:
    args = pickle.load(f)

# CAREFUL with this -->
if not hasattr(args, 'normalization_factor'):
    args.normalization_factor = 1
if not hasattr(args, 'aggregation_method'):
    args.aggregation_method = 'sum'

args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if args.cuda else "cpu")
args.device = device
dtype = torch.float32
utils.create_folders(args)
print(args)

dataset_info = get_dataset_info(args.dataset, args.remove_h)

print(dataset_info)

dataloaders, charge_scale = dataset.retrieve_dataloaders(args)

flow, nodes_dist, prop_dist = get_model(
    args, device, dataset_info, dataloaders['train'])
flow.to(device)

fn = 'generative_model_ema.npy' if args.ema_decay > 0 else 'generative_model.npy'
flow_state_dict = torch.load(join(eval_args.model_path, fn),
                                map_location=device)

flow.load_state_dict(flow_state_dict)

outputs/edm_qm9\args.pickle
Namespace(actnorm=True, aggregation_method='sum', attention=True, augment_noise=0, batch_size=64, break_train_epoch=False, brute_force=False, clip_grad=True, condition_time=True, conditioning=[], context_node_nf=0, cuda=True, current_epoch=1001, data_augmentation=False, datadir='qm9/temp', dataset='qm9', dequantization='argmax_variational', device=device(type='cuda'), diffusion_loss_type='l2', diffusion_noise_precision=1e-05, diffusion_noise_schedule='polynomial_2', diffusion_steps=1000, dp=True, ema_decay=0.9999, exp_name='polynomial_2_final_0', filter_n_atoms=None, generate_epochs=1, include_charges=True, inv_sublayers=1, lr=0.0001, model='egnn_dynamics', n_epochs=3000, n_layers=9, n_report_steps=1, n_stability_samples=1000, nf=256, no_cuda=False, no_wandb=False, norm_constant=1, normalization_factor=1, normalize_factors=[1, 4, 10], num_workers=0, ode_regularization=0.001, online=True, probabilistic_model='diffusion', remove_h=False, resume=None, save_mode

<All keys matched successfully>

In [30]:
import site
print(site.getsitepackages())

['c:\\Users\\crist\\anaconda3\\envs\\gaudi', 'c:\\Users\\crist\\anaconda3\\envs\\gaudi\\lib\\site-packages']


In [41]:
print('Sampling handful of molecules.')
# sample_different_sizes_and_save(
#     args, eval_args, device, flow, nodes_dist,
#     dataset_info=dataset_info, n_samples=29)

print('Sampling stable molecules.')
sample_only_stable_different_sizes_and_save(
    args, eval_args, device, flow, nodes_dist,
    dataset_info=dataset_info, n_samples=3, n_tries=2*10)

Sampling handful of molecules.
Sampling stable molecules.


  1%|          | 9/1000 [00:48<1:29:54,  5.44s/it]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\crist\anaconda3\envs\gaudi\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\crist\AppData\Local\Temp\ipykernel_29344\1166628403.py", line 7, in <module>
    sample_only_stable_different_sizes_and_save(
  File "d:\Projects\e3_diffusion_for_molecules\eval_sample.py", line 68, in sample_only_stable_different_sizes_and_save
    one_hot, charges, x, node_mask = sample(
  File "d:\Projects\e3_diffusion_for_molecules\qm9\sampling.py", line 139, in sample
    x, h = generative_model.sample(batch_size, max_n_nodes, node_mask, edge_mask, context, fix_noise=fix_noise, dataset_info = dataset_info)
  File "c:\Users\crist\anaconda3\envs\gaudi\lib\site-packages\torch\autograd\grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "d:\Projects\e3_diffusion_for_molecules\equivariant_diffusion\en_diffusion.py", line 864, in sam

In [40]:
vis.visualize(
    join(eval_args.model_path, 'eval/molecules/'), dataset_info,
    max_num=100, spheres_3d=True)

In [ ]:
print('Sampling visualization chain.')
save_and_sample_chain(
    args, eval_args, device, flow,
    n_tries=eval_args.n_tries, n_nodes=eval_args.n_nodes,
    dataset_info=dataset_info)